## COVID daily case count data integrity issue

This notebook explores a data integrity issue with the COVID daily case count data. The rate at which actual infections have been confirmed with a case diagnosis has not been constant throughout the course of the pandemic. Undercounting was most acute during the early stages of the pandemic. This article demonstrates how the well-known, but not necessarily well-understood, data integrity issue of undercounting can impact time period comparisons of the daily antigen case diagnosis data.

The point of illustration centers around a chart that appeared in the New York Times on February 7, 2021. This chart was recreated. Three similar charts were created for comparison using the same data except for changes that were made to try to adjust for the data integrity issue.

For further discussion, see the file [undercount_discussion.md](undercount_discussion.md).

## Tools and techniques used in this project
- **Tools**
> - Python, Pandas, Numpy, Markdown
- **Visualization**
> - Matplotlib, Plotly
- **Techniques**
> - Datetime, Data Integrity Evaluation, Simple Moving Average, Linear Regression



In [31]:
import pandas as pd
import numpy as np
import datetime
from datetime import timedelta
import matplotlib.pyplot as plt

In [32]:
from urllib.request import urlopen
import plotly.express as px
import json
with urlopen('https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json') as response:
    counties = json.load(response)


## Load and prep data tables

In [33]:
covid_df = pd.read_csv('https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-counties.csv', dtype={"fips": str}, parse_dates=['date'])

In [ ]:
# Create a dataframe for counties which will be used later.
counties_df = covid_df.groupby(['fips', 'county', 'state']).agg(FIPS = ('fips', 'max'),
                                                              COUNTY = ('county', 'max'),
                                                              ST = ('state', 'max')
                                                             ).reset_index()[['FIPS', 'COUNTY', 'ST']]

In [ ]:
# NYT covid cases are cumulative. Calculate incremental new cases for each county (fips).
covid_df['daily_cases'] = covid_df.groupby('fips')['cases'].diff().fillna(0)

In [ ]:
# Let's recalibrate the case numbers a few ways to attempt to correct for undercount rate. 
covid_df['daily_cases_low'] = np.where(covid_df['date']<'2020-06-01', covid_df['cases_new']*4, 
                                     covid_df['cases_new']*2.1)

covid_df['daily_cases_med'] = np.where(covid_df['date']<'2020-06-01', covid_df['cases_new']*5.5, 
                                     covid_df['cases_new']*2.1)

covid_df['daily_cases_high'] = np.where(covid_df['date']<'2020-06-01', covid_df['cases_new']*7, 
                                     covid_df['cases_new']*2.1)

In [ ]:
def safe_div(x, y):
    try:
        return x/y
    except ZeroDivisionError:
        return 0

In [ ]:
def add_sma_and_ratio(cvd_df, case_column):
    '''
    First, calculate the simple moving average (SMA) for each county (fips).
    Next, calculate the datapoints that are used in the NYT charts--SMA peak prior 
    to Oct 1 and weekly average prior to Feb 6, 2021 (I am using ave of SMA).
    Parameters:
    cvd_df: pandas dataframe
    case_column: column name of the pandas dataframe to use as input
    Returns:
    Pandas series of the computed current to peak ratio per county
    '''
    county_SMA = []
    for fips_ in cvd_df['fips'].unique():
        county_df = cvd_df[cvd_df['fips']==fips_]
        county_SMA.append(county_df[case_column].rolling(window=7).mean())
    cvd_df[case_column] = pd.concat(county_SMA)

    county_prev_peak, county_curr_ave = [], []
    for fips_ in counties_df['FIPS'].unique():
        county_prev_peak.append(cvd_df[(cvd_df['fips']==fips_) & (cvd_df['date'] < '2020-10-01')][case_column].max())
        county_curr_ave.append(cvd_df[(cvd_df['fips']==fips_) & (cvd_df['date'] > '2020-10-01') & 
                                   (cvd_df['date'] < '2021-02-06')][case_column][-7:].mean())

    return [safe_div(county_curr_ave[i], county_prev_peak[i]) for i in range(len(county_curr_ave))]
    

In [ ]:
counties_df['Curr to Peak'] = add_sma_and_ratio(covid_df, 'daily_cases')
counties_df['Curr to Peak: Low Recalibration'] = add_sma_and_ratio(covid_df, 'daily_cases_low')
counties_df['Curr to Peak: Med Recalibration'] = add_sma_and_ratio(covid_df, 'daily_cases_med')
counties_df['Curr to Peak: High Recalibration'] = add_sma_and_ratio(covid_df, 'daily_cases_high')

# The above calculations take a while to run. It's best to export the CSV to save it and then
# import it again when needed.
counties_df.to_csv('../data/covid_curr_to_peak_comparison_3x.csv')

In [16]:
counties_df = pd.read_csv('../data/covid_curr_to_peak_comparison_3x.csv', dtype={"FIPS": str})

In [26]:
def make_comparative_chart(df, data_column, title, footnote_1, footnote_2, img_file_root):
    '''
    Construct a Plotly chloropleth US map at the county level comparing the current case rate to the peak rate
    prior to 10/1/20. 
    Parameters:
    -----------
    df: input dataframe
    data_column: data column name
    title, footnote_1, footnote_2, img_file_root: strings
    Returns:
    --------
    NA
    '''
    fig = px.choropleth_mapbox(df, geojson=counties, locations='FIPS', color=data_column,
                               color_continuous_scale = ['green', 'yellow', 'red'],
                               range_color=(0, 3),
                               mapbox_style="carto-positron",
                               zoom=2.8, center = {"lat": 37.0902, "lon": -95.7129},
                               opacity=0.3,
                               labels ={data_column: 'Curr to Peak'}
                              )
    fig.update_layout(margin={"r":0,"t":25,"l":0,"b":25},
                    title_text = title,
                    geo_scope='usa', # limit map scope to USA
                    annotations = [dict(x=0.55, y=0.065, text=footnote_1, showarrow = False), 
                               dict(x=0.55, y=0.025, text=footnote_2, showarrow = False)]
                    )
    fig.write_html(f'img/{img_file_root}.html', auto_open=False)
    fig.write_image(f"img/{img_file_root}.png")


In [27]:
make_comparative_chart(df=counties_df, data_column='Curr to Peak', 
                       title='Average daily new coronavirus cases (uncalibrated data) compared with previous peak',
                      footnote_1='Original Source: New York Times database of reports from state and local health agencies',
                      footnote_2=' ',
                      img_file_root='curr_vs_peak_covid')

In [28]:
make_comparative_chart(df=counties_df, data_column='Curr to Peak: Low Recalibration', 
                       title='Average daily new coronavirus cases (low recalibration) compared with previous peak',
                      footnote_1='Original Source: New York Times database of reports from state and local health agencies',
                      footnote_2='Modifications based on CDC National Commercial Laboratory Seroprevalence Survey',
                      img_file_root='low_recal_curr_vs_peak_covid')

In [29]:
make_comparative_chart(df=counties_df, data_column='Curr to Peak: Med Recalibration', 
                       title='Average daily new coronavirus cases (med. recalibration) compared with previous peak',
                      footnote_1='Original Source: New York Times database of reports from state and local health agencies',
                      footnote_2='Modifications based on CDC National Commercial Laboratory Seroprevalence Survey',
                      img_file_root='med_recal_curr_vs_peak_covid')

In [30]:
make_comparative_chart(df=counties_df, data_column='Curr to Peak: High Recalibration', 
                       title='Average daily new coronavirus cases (high recalibration) compared with previous peak',
                      footnote_1='Original Source: New York Times database of reports from state and local health agencies',
                      footnote_2='Modifications based on CDC National Commercial Laboratory Seroprevalence Survey',
                      img_file_root='high_recal_curr_vs_peak_covid')